## 난임 환자 대상 임신 성공 여부 예측

### LGAimers 6th 온라인 해커톤

Import

In [1]:
import pandas as pd
import optuna
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import lightgbm as lgb

### Data Load

In [2]:
# 데이터 로드
Total_train = pd.read_csv('../data/Total_train_dataset_51.csv')
Total_test = pd.read_csv('../data/Total_test_dataset_51.csv')

In [3]:
# ID 열을 제외한 특성과 타겟 변수 분리
Total_X = Total_train.drop(['임신_성공_여부', 'ID'], axis=1)
Total_y = Total_train['임신_성공_여부']

### 인코딩 

In [4]:
Total_categorical_columns = [
    # 없음
]

In [5]:
# 모든 범주형 변수를 문자열로 변환
Total_X[Total_categorical_columns] = Total_X[Total_categorical_columns].astype(str)
Total_test[Total_categorical_columns] = Total_test[Total_categorical_columns].astype(str)

# OrdinalEncoder를 사용하여 범주형 변수 인코딩
Total_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

Total_X[Total_categorical_columns] = Total_encoder.fit_transform(Total_X[Total_categorical_columns])
Total_test[Total_categorical_columns] = Total_encoder.transform(Total_test[Total_categorical_columns])

## Modeling

In [6]:
# 데이터 분할
Total_X_train, Total_X_test, Total_y_train, Total_y_test = train_test_split(Total_X, Total_y, test_size=0.2, random_state=42)

### Total 데이터

optuna

In [7]:
# 목적 함수 정의
def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 500, 8000),
        'num_leaves': trial.suggest_int('num_leaves', 100, 5000),
        'max_depth': trial.suggest_int('max_depth', 5, 500),
        'learning_rate': trial.suggest_float('learning_rate', 0.0005, 0.5),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 500),
        'subsample': trial.suggest_float('subsample', 0.0, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.01, 100.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 100.0),

        'random_state': 42,
        'n_jobs': -1,
        'metric': 'auc',
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'verbose': -1
    }

    model = lgb.LGBMClassifier(**param)
    model.fit(Total_X_train, Total_y_train)
    
    y_pred_proba = model.predict_proba(Total_X_test)[:, 1]
    
    auc = roc_auc_score(Total_y_test, y_pred_proba)
    return auc

# Optuna 스터디 생성 및 최적화 실행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# 최적의 하이퍼파라미터 출력
print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2025-02-23 23:23:23,757] A new study created in memory with name: no-name-71952252-a5ac-44bc-9025-3f9d7a484469
[I 2025-02-23 23:23:44,927] Trial 0 finished with value: 0.7159716912827899 and parameters: {'n_estimators': 6491, 'num_leaves': 793, 'max_depth': 318, 'learning_rate': 0.2561371598439815, 'min_child_samples': 356, 'subsample': 0.43043377236337266, 'reg_alpha': 5.7592392074464565, 'reg_lambda': 54.29197165060359}. Best is trial 0 with value: 0.7159716912827899.
[I 2025-02-23 23:23:47,839] Trial 1 finished with value: 0.737540641408274 and parameters: {'n_estimators': 1071, 'num_leaves': 1840, 'max_depth': 252, 'learning_rate': 0.3368065575500847, 'min_child_samples': 478, 'subsample': 0.6131815831095374, 'reg_alpha': 26.72973676770004, 'reg_lambda': 50.65635078632265}. Best is trial 1 with value: 0.737540641408274.
[I 2025-02-23 23:23:54,822] Trial 2 finished with value: 0.7369503001364934 and parameters: {'n_estimators': 2545, 'num_leaves': 953, 'max_depth': 496, 'learning

KeyboardInterrupt: 

.